# Qaintellect Training Example

In this example, we will demonstrate the optimisation of a quantum circuit using Qaintellect's Flux integration.

In [1]:
include("../src/Qaintellect.jl")
using .Qaintellect

using Flux, LinearAlgebra, Test
using Flux.Optimise: update!
using IterTools: ncycle

## Task: Creating a Flux layer using Qaintessent
Here, we create a complex circuit with $N$ qubits and different input parameters. We then train these parameters using the Flux framework.

In [2]:
# construct parametrized circuit
N = 4
rz = RzGate(1.5π)
ps = PhaseShiftGate(0.3)
ry = RyGate(√2)
n = randn(Float64, 3)
n = n/norm(n)
rg = RotationGate(0.2π, n)

cgc = CircuitGateChain{N}([
    single_qubit_circuit_gate(3, HadamardGate(), N),
    controlled_circuit_gate((1, 4), 2, rz, N),
    two_qubit_circuit_gate(2, 3, SwapGate(), N),
    single_qubit_circuit_gate(3, ps, N),
    single_qubit_circuit_gate(3, rg, N),
    single_qubit_circuit_gate(1, ry, N),
])
meas = MeasurementOps{N}([Matrix{Float64}(I, 2^N, 2^N), Hermitian(randn(ComplexF64, 2^N, 2^N))])

c = Circuit(cgc, meas)


    1 ————————•————————————————[Ry]—
              |                     
    2 ———————[Rz]———x———————————————
              |     |               
    3 —[H ]—————————x————[Pϕ]——[Rθ]—
              |                     
    4 ————————•—————————————————————


In [3]:
function output(ψ, Δ)
    @assert length(ψ) == length(Δ)
    dot(ψ, Δ)
end

output (generic function with 1 method)

In [4]:
# input quantum state
ψ = randn(ComplexF64, 2^N)
ψ = ψ/norm(ψ)

# fictitious gradients of cost function with respect to circuit output
Δ = [0.3, -1.2]
e = 0.65

0.65

In [5]:
# set up model
model(ψ) = output(c(ψ), Δ)

# create loss function
loss(x,y) = Flux.mse(model(x), y)

# gather parameters from Circuit
paras = Flux.params(c)

# freeze parameter ry.θ, equivalent to delete!(paras, Qaintessent.get_trainable(ry))
delete!(paras, ry.θ)

# set up data for training
data = ncycle([(ψ, e)], 150)

# define optimizer
opt = Descent(1)

# define evaluation function
evalcb() = @show(loss(ψ, e))

evalcb (generic function with 1 method)

In [6]:
println("Initial model evaluation: " * string(model(ψ)) * ", Target: " * string(e))

Flux.train!(loss, paras, data, opt, cb=Flux.throttle(evalcb, 0.01))

println("Final model evaluation: " * string(model(ψ)) * ", Target: " * string(e))

Initial model evaluation: 0.81231033534757, Target: 0.65
loss(ψ, e) = 0.06788626908178497
loss(ψ, e) = 0.08252209923351557
loss(ψ, e) = 0.021491255499266927
loss(ψ, e) = 0.008243733611891345
loss(ψ, e) = 0.0016683438797921654
loss(ψ, e) = 0.0003959080660997218
loss(ψ, e) = 7.44698535985435e-5
loss(ψ, e) = 2.110911877948835e-5
loss(ψ, e) = 4.635790663946177e-6
loss(ψ, e) = 1.0067283253589037e-6
loss(ψ, e) = 2.1974857421217475e-7
loss(ψ, e) = 5.4987657431347064e-8
loss(ψ, e) = 1.043141900149684e-8
loss(ψ, e) = 2.998292398943378e-9
Final model evaluation: 0.6500293603081675, Target: 0.65
